In [1]:
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
gparentdir = os.path.dirname(os.path.dirname(currentdir))
sys.path.insert(0, gparentdir)
import scobra
import get_all_functions as gaf
import re

new=scobra.Model('sample/12018_0_supp_510122_p0t431.xls')
new= scobra.model(new)

In [2]:
new.Reactions()

['+-BORNEOL-DEHYDROGENASE-RXN_c',
 '+-NEOMENTHOL-DEHYDROGENASE-RXN_c',
 '--MENTHOL-DEHYDROGENASE-RXN_c',
 '1.1.1.141-RXN_c',
 '1.1.1.170-RXN-NADP_c',
 '1.1.1.170-RXN-NAD_c',
 '1.1.1.178-RXN_m',
 '1.1.1.210-RXN_c',
 '1.1.1.222-RXN-NADP_c',
 '1.1.1.222-RXN-NAD_c',
 '1.1.1.248-RXN_c',
 '1.1.1.255-RXN_c',
 '1.1.1.268-RXN_c',
 '1.1.1.271-RXN_c',
 '1.1.1.285-RXN-NADP_c',
 '1.1.1.285-RXN-NAD_c',
 '1.1.1.288-RXN_c',
 '1.1.1.34-RXN_c',
 '1.1.1.39-RXN_c',
 '1.1.1.51-RXN-NADP_c',
 '1.1.1.51-RXN-NAD_c',
 '1.1.1.64-RXN_c',
 '1.1.1.8-RXN_c',
 '1.1.3.37-RXN_c',
 '1.10.2.2-RXN_m',
 '1.14.11.18-RXN_c',
 '1.14.11.20-RXN_c',
 '1.14.13.117-RXN_c',
 '1.14.13.118-RXN_c',
 '1.14.13.36-RXN_c',
 '1.14.13.42-RXN_c',
 '1.14.13.70-RXN_c',
 '1.14.13.72-RXN-NADP_c',
 '1.14.13.72-RXN-NAD_c',
 '1.14.13.73-RXN_c',
 '1.14.13.77-RXN_c',
 '1.14.13.78-RXN_c',
 '1.14.13.79-RXN_c',
 '1.14.13.8-RXN_c',
 '1.14.13.87-RXN-NADP_c',
 '1.14.13.87-RXN-NAD_c',
 '1.14.13.93-RXN_c',
 '1.14.21.6-RXN-NADP_c',
 '1.14.21.6-RXN-NAD_c',
 '1

In [3]:
for i in new.Reactions():
    if new.GetReaction(i).lower_bound == 0.0:
        new.GetReaction(i).upper_bound == 1000.0
    else:
        new.GetReaction(i).lower_bound == 1000.0
        new.GetReaction(i).upper_bound == 1000.0
new.SetConstraint('O2_pc', -1, -1)
new.SetObjective('Photon_tx')
new.SetObjDirec('Max')
new.Solve()

optimal


In [4]:
len(new.GetSol())

61

In [5]:
#Maximize Sucrose output (objective)
#vary light (scan)
#allow CO2 and O2, even Proton and Water
#Output maintenance ATP and NADPHox
tx_and_biomass = []
for i in new.Reactions(): 
    if '_tx' in i:
        tx_and_biomass.append(re.search('(.*)_tx', i).group())
    elif '_biomass' in i: 
        tx_and_biomass.append(re.search('(.*)_biomass', i).group())
#tx_and_biomass.append(re.search('(.*)_tx', 'slkdfjd_tx').group())
tx_and_biomass

necessary_rxn= ['CO2_tx', 'Photon_tx', 'O2_tx','H2O_tx','NADPHoxc_tx','NADPHoxm_tx','NADPHoxp_tx','Starch_tx']
unecessary_rxn= list(set(tx_and_biomass) - set(necessary_rxn))

#blocking _tx and _biomass reactions that are not considered
for i in unecessary_rxn: 
    new.GetReaction(i).lower_bound = 0.0
    new.GetReaction(i).upper_bound = 0.0

#freeing _tx and _biomass reactions that are considered
for i in necessary_rxn: 
    if new.GetReaction(i).lower_bound == 0.0:
        new.GetReaction(i).upper_bound == 1000.0
    else: 
        new.GetReaction(i).lower_bound == 1000.0
        new.GetReaction(i).upper_bound == 1000.0

#setting constraints
#new.SetConstraint('ATPase_tx', 90, 90)
#new.SetConstraint('NADPHoxc_tx', 10, 10)
#new.SetConstraint('NADPHoxm_tx', 10, 10)
#new.SetConstraint('NADPHoxp_tx', 10, 10)
#setting constraints for sucrose entering and exiting cytoplasm
new.SetConstraint('Starch_tx', -1, -1)

#setting objectives
new.SetObjective('Photon_tx')
new.SetObjDirec('Min')
#new.SetFixedFlux({'unlProtHYPO_c': 0})

#running scans
#new.MinFluxSolve() 
new.Solve()

optimal


In [11]:
import timeit
start = timeit.default_timer()
new.MinFluxSolve()
end = timeit.default_timer()
time_taken = end - start

optimal


In [15]:
time_taken / 60

17.217797799905142

In [16]:
len(new.GetSol())

43

In [4]:
tx_and_biomass = []
for i in new.Reactions(): 
    if '_tx' in i:
        tx_and_biomass.append(re.search('(.*)_tx', i).group())
new.PrintReactions(tx_and_biomass)

ATP_tx	 --> ATP_p
ATPase_tx	ADP_c + 2.0 PROTON_c + Pi_c --> ATP_c + WATER_c
CO2_tx	 <=> CARBON-DIOXIDE_c
Ca_tx	Pumped-PROTON_c --> CA+2_c + PROTON_c
Fe_tx	 --> IRON(II)_c
GLC_tx	Pumped-PROTON_c --> GLC_c + PROTON_c
H2O_tx	 <=> WATER_c
K_tx	Pumped-PROTON_c --> K+_c + PROTON_c
Mg_tx	Pumped-PROTON_c --> MG+2_c + PROTON_c
NADPHoxc_tx	NADP_c + WATER_c <=> NADPH_c + 0.5 OXYGEN-MOLECULE_c + PROTON_c
NADPHoxm_tx	NADP_m + WATER_m <=> NADPH_m + 0.5 OXYGEN-MOLECULE_m + PROTON_m
NADPHoxp_tx	NADP_p + WATER_p <=> NADPH_p + 0.5 OXYGEN-MOLECULE_p + PROTON_p
NADPHoxx_tx	NADP_x + WATER_x --> NADPH_x + 0.5 OXYGEN-MOLECULE_x + PROTON_x
NH4_tx	 --> AMMONIA_c + PROTON_c
NO3_tx	2.0 Pumped-PROTON_c --> NITRATE_c + 2.0 PROTON_c
O2_tx	 <=> OXYGEN-MOLECULE_c
PROTON_tx	 --> PROTON_c
Photon_tx	 --> Photon_p
Pi_tx	3.0 Pumped-PROTON_c --> 3.0 PROTON_c + Pi_c
SO4_tx	3.0 Pumped-PROTON_c --> 3.0 PROTON_c + SULFATE_c
Starch_tx	 --> STARCH_p
Sucrose_tx	Pumped-PROTON_c <-- PROTON_c + SUCROSE_c
TAG_tx	 --> GLYCEROL_c + 3.0

In [3]:
new.GenesToSubsystemsAssociations

{'SOLYC06G063220.2': ['glycolysis I'],
 'SOLYC06G050980.2': [],
 'SOLYC08G048450.2': ['citrulline biosynthesis'],
 'SOLYC04G082180.2': ['pyruvate fermentation to ethanol II',
  'UDP-D-xylose biosynthesis'],
 'SOLYC07G064810.2': ['histidine biosynthesis'],
 'SOLYC12G044600.2': ['gluconeogenesis'],
 'SOLYC10G085360.1': ['sucrose degradation III'],
 'SOLYC10G086730.1': ['glycolysis I'],
 'SOLYC03G013460.1': ['aerobic respiration -- electron donor II'],
 'SOLYC03G065340.2': ['starch degradation'],
 'SOLYC09G082970.2': ['Rubisco shunt'],
 'SOLYC00G025510.1': ['serine biosynthesis'],
 'SOLYC08G083110.2': ['homocysteine and cysteine interconversion'],
 'SOLYC03G121070.2': ['sucrose degradation III', 'glycogen degradation I'],
 'SOLYC03G121420.2': [],
 'SOLYC01G100630.1': ['removal of superoxide radicals',
  'oxidative ethanol degradation III'],
 'SOLYC11G039760.1': ['starch biosynthesis'],
 'SOLYC07G053150.2': [],
 'SOLYC12G088160.1': ['gluconeogenesis'],
 'SOLYC11G013810.1': ['nitrate reduct

In [17]:
lst_submodel = [new.GetReaction(x) for x in new.GetSol()]
testm = new.SubModel(lst_submodel)

print( [ i for i in testm.Reactions() if 'ATP' in i])

['Plastidial_ATP_Synthase_p', 'ATP_ADP_Pi_pc']


In [20]:
testm.Reactions()

['GAPOXNPHOSPHN-RXN_c',
 'GLYCOGENSYN-RXN_p',
 'PHOSPHOGLUCMUT-RXN_p',
 'GAP_3PGA_pc',
 'Plastidial_ATP_Synthase_p',
 'RXN1F-66_p',
 'RXN-7677_p',
 'PGLUCISOM-RXN_p',
 'PHOSPHORIBULOKINASE-RXN_p',
 'PHOSGLYPHOS-RXN_c',
 'RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_p',
 'OAA_MAL_pc',
 'RXN-7678-NAD_p',
 'NADPH_Dehydrogenase_p',
 '1.18.1.2-RXN_p',
 'SEDOHEPTULOSE-BISPHOSPHATASE-RXN_p',
 'TRIOSEPISOMERIZATION-RXN_p',
 'RXN-7674_p',
 'NADPHoxp_tx',
 'F16BDEPHOS-RXN_p',
 'Plastoquinol_Oxidase_p',
 'ATP_ADP_Pi_pc',
 'SEDOBISALDOL-RXN_p',
 'CO2_pc',
 'RXN-7679_p',
 'O2_pc',
 'F16ALDOLASE-RXN_p',
 '1TRANSKETO-RXN_p',
 'NADPHoxc_tx',
 'GLUC1PADENYLTRANS-RXN_p',
 'PYRNUTRANSHYDROGEN-RXN_c',
 'H2O_tx',
 'H2O_pc',
 'RIBULP3EPIM-RXN_p',
 'CO2_tx',
 '2TRANSKETO-RXN_p',
 'O2_tx',
 'RXN-7676_p',
 'MALATE-DEH-RXN_c',
 'Starch_tx',
 'RIB5PISOM-RXN_p',
 'MALATE-DEH-RXN_p',
 'INORGPYROPHOSPHAT-RXN_p']

In [21]:
testm.SetObjective('CO2_tx')
testm.SetObjDirec('Min')

In [22]:
testm.Solve()
testm.GetSol()
len(testm.GetSol())

optimal


43

In [23]:
testm.WriteModel('/home/kristoforus/compbio/scobra/TestPrograms/sample/testmodel2.xls')